In [2]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import sklearn 



In [3]:
#LOAD GIS WATERMAIN AND BREAK DATA
df = pd.read_csv('Kirk_All.csv')
#df.dropna(inplace=True) # Are there any columns where we can deal with NA values?
# Can we use imputation instead?
# breaks = pd.read_csv('Kirk_Break_Data.csv')
df['INSTYEAR'] = df['INSTYEAR'].map(lambda x: int((str(x)[-12:-8])), na_action='ignore')
#df = pd.get_dummies(df)
df.head()

,FID,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,6.0,,,


In [4]:
# EXTRACTING FEATURES
features = ['FID_WA_Mai', 'TYPE', 'LENGTH', 'DIAMETER', 'MATERIAL', 'INSTYEAR', 'MUSYM', 'Dist_Rail', 'FULL_STRNA' , 'SLP_CLASS', 'SPEED', 'FUNC_CLASS']
pipes = df[features]
pipes

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SLP_CLASS,SPEED,FUNC_CLASS
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,2.0,25,MINOR ARTERIAL
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,122ND LN NE,1.0,25,NEIGHBORHOOD ACCESS
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,0.0,35,PRINCIPAL ARTERIAL
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,LAKE WASHINGTON BLVD NE,0.0,35,PRINCIPAL ARTERIAL
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,,6.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
41816,3963,DIST,1.0,12.0,DI,1971.0,No,9000,120TH AVE NE,0.0,25,COLLECTOR
41817,5622,DIST,1.0,8.0,DI,2003.0,AgC,10000,117TH AVE NE,2.0,25,NEIGHBORHOOD ACCESS
41818,5623,DIST,1.0,8.0,DI,2003.0,AgC,10000,117TH AVE NE,2.0,25,NEIGHBORHOOD ACCESS
41819,1093,DIST,83.0,12.0,DI,2009.0,AgC,5000,NE 115TH PL,0.0,35,COLLECTOR


In [5]:
breaks

NameError: name &#39;breaks&#39; is not defined

In [ ]:
# GROUP PIPES BY THEIR UNIQUE ID - NEEDED BECAUSE OF HOW SLOPE BREAKS UP PIPES IN DIFFERENT SLOPE CLASSES
def group_by_id(df):

    grouping = df.groupby('FID_WA_Mai').groups
    unique_pipes = df['FID_WA_Mai'].unique()
    indices = []
    for i in range(len(unique_pipes)):
        pipe = unique_pipes[i]
        val = grouping[pipe][0]
        indices.append(val)
    
    # CALCULATE AVERAGE SLOPES FOR PIPES THAT GOT SPLIT INTO MULTIPLE ROWS
    slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
    slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
    slopes = pd.Series(slopes)

    filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
    filtered = filtered.drop('SLP_CLASS', axis=1)
    filtered['SLOPE'] = slopes.values

    return filtered

In [ ]:
pipes = group_by_id(pipes)
pipes

In [135]:
break_1_8_04 = filtered[(filtered['FULL_STRNA'] == '7TH AVE') & (filtered['INSTYEAR'].equals(2003))]
break_1_8_04

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE


In [204]:
break_1_9_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W')] #& (filtered['INSTYEAR'] < 2005) & (filtered['INSTYEAR'] > 2003)]
break_1_9_04 = break_1_9_04[break_1_9_04['INSTYEAR'] == 2004]
break_1_9_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004) & (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 8.0)] 
nearby

similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [205]:
break_1_21_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] == 2004)]
break_1_21_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004)] #& (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 6.0)] 


similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [197]:
break_2_25_04 = filtered[(filtered['FULL_STRNA'] == 'NE 59TH ST')]
break_2_25_04

# THIS WILL LIKELY NEED TO BE altered in data set

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22011,3634,DIST,1.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
22012,3635,DIST,5.0,12.0,AC,1971,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
39310,6927,DIST,261.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,2.586207


In [9]:
# Find pipes on NE 73RD ST installed in 2008
break_1_24_08 = pipes[(pipes['FULL_STRNA'] == 'NE 73RD ST') & (pipes['INSTYEAR'] == 2008)] 
# Find pipes installed before 2008 with similar attributes to find when broken pipes was installed
near1 = pipes[(pipes['FULL_STRNA'] == 'NE 73RD ST') & (pipes['INSTYEAR'] < 2008) & (pipes["Dist_Rail"] == 8000) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC")]
# Two possibilities 1959 and 1963, find the most common year and use that 
search59 = pipes[(pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["INSTYEAR"] == 1959)]
search63 = pipes[(pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["INSTYEAR"] == 1963)]
print(break_1_24_08)
print(search63)
# End with using 1963 as install year

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
22567        3719  DIST     3.0       8.0       DI    2008.0   AgC       8000   
22568        3720  DIST     4.0       8.0       DI    2008.0   AgC       8000   
23946        3992  DIST    15.0       8.0       DI    2008.0   AgC       8000   
23948        3993  DIST     3.0       8.0       DI    2008.0   AgC       8000   
26885        4497  DIST     3.0       8.0       DI    2008.0   AgC       8000   
28129        4706  DIST   102.0       8.0       DI    2008.0   AgC       8000   
30433        5100  DIST   482.0       8.0       DI    2008.0   AgC       8000   

       FULL_STRNA SPEED           FUNC_CLASS     SLOPE  
22567  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
22568  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
23946  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
23948  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
26885  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.642857  
28129  NE

In [27]:
# No pipes installed on 9th ave in year of break, went to 2009
# all of these pipes have a larger diameter and different material than pipe that broke
break_1_29_08 = pipes[(pipes['FULL_STRNA'] == '9TH AVE') & (pipes['INSTYEAR'] == 2009 )]  
break_1_29_08
# Find pipes on 9th ave built before 2008 with similar attributes to the broken pipe. No pipes have same diameter
# Only one pipe where bottom line is true, more if remove Dist_Rail, all of these pipes intalled in 1968
near2 =  pipes[(pipes['FULL_STRNA'] == '9TH AVE') & (pipes['INSTYEAR'] < 2008)  & (pipes["MATERIAL"] == "AC") & (pipes["Dist_Rail"] == 12000)]
print(break_1_29_08)
near2

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
15914        2593  DIST     4.0      12.0       DI    2009.0   AmC      12000   
15915        2594  DIST     4.0      12.0       DI    2009.0   AmC      12000   
16042        2606  DIST     4.0      12.0       DI    2009.0   AmC      12000   
23601        3893  DIST     4.0      12.0       DI    2009.0   AmC      12000   
24474        4073  DIST     4.0      12.0       DI    2009.0   AmC      12000   
24507        4078  DIST    59.0      12.0       DI    2009.0   AmC      12000   
24510        4079  DIST   415.0      12.0       DI    2009.0   AmC      12000   
24527        4081  DIST     9.0      12.0       DI    2009.0   AmC      12000   

      FULL_STRNA SPEED           FUNC_CLASS  SLOPE  
15914    9TH AVE    25  NEIGHBORHOOD ACCESS    1.0  
15915    9TH AVE    25  NEIGHBORHOOD ACCESS    1.0  
16042    9TH AVE    25  NEIGHBORHOOD ACCESS    1.0  
23601    9TH AVE    25  NEIGHBORHOOD ACCESS    1.0  
24474

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
16008,2604,DIST,555.0,10.0,AC,1968.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.277778


In [45]:
#No pipes installed on 4th st until about 2015 which is weird
break_2_9_08 = pipes[(pipes["FULL_STRNA"] == "4TH ST") & (pipes["INSTYEAR"] > 2008)]
break_2_9_08
# Find pipes on 4th st built before 2008 with similar attributes to the broken pipe.
near3 = pipes[(pipes["FULL_STRNA"] == "4TH ST") & (pipes["INSTYEAR"] < 2008) & (pipes["MATERIAL"] == "AC") & (pipes["Dist_Rail"] == 13000) & (pipes["DIAMETER"] == 6)]
near3
#End with 1968 as install year

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
16075,2616,DIST,584.0,6.0,AC,1968.0,AmC,13000,4TH ST,25,NEIGHBORHOOD ACCESS,1.125
16132,2624,DIST,1.0,6.0,AC,1968.0,AmC,13000,4TH ST,25,NEIGHBORHOOD ACCESS,2.000
35103,5906,DIST,379.0,6.0,AC,1968.0,AmC,13000,4TH ST,25,NEIGHBORHOOD ACCESS,2.000
39867,7107,DIST,13.0,6.0,AC,1968.0,AmC,13000,4TH ST,25,NEIGHBORHOOD ACCESS,1.000


In [63]:
#There are pipes that were installed on 85th st in 2008, however their distance from rail is different from pipes in near4
break_2_18_08 = pipes[(pipes["FULL_STRNA"] == "NE 85TH ST") & (pipes["INSTYEAR"] == 2008)] 
print(break_2_18_08)
#Find pipes on 85th st installed before 2008 with similar attributes
#Very likely that the broken pipe was installed with these pipes but the dist_rail is different from pipes
#that were installed in 2008, closer in dist_rail value to pipes installed more recently than 2008
near4 = pipes[(pipes["FULL_STRNA"] == "NE 85TH ST") & (pipes["INSTYEAR"] < 2008) & (pipes["MATERIAL"] == "CONC") & (pipes["DIAMETER"] == 16)]
near4
#End with 1970 as install year

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
24310        4046  DIST   589.0       8.0       DI    2008.0   AgC       6000   
24334        4047  DIST     3.0       8.0       DI    2008.0   AgC       6000   
24335        4048  DIST     4.0       8.0       DI    2008.0   AgC       6000   
24433        4062  DIST    24.0       4.0       DI    2008.0   AgC       6000   
28215        4723  DIST   384.0       8.0       DI    2008.0   AgC       6000   
33573        5618  DIST     4.0       8.0       DI    2008.0   AgC       6000   
33574        5619  DIST    76.0       8.0       DI    2008.0   AgC       6000   
33578        5620  DIST     4.0       8.0       DI    2008.0   AgC       6000   

       FULL_STRNA SPEED          FUNC_CLASS     SLOPE  
24310  NE 85TH ST    35  PRINCIPAL ARTERIAL  0.750000  
24334  NE 85TH ST    35  PRINCIPAL ARTERIAL  0.916667  
24335  NE 85TH ST    35  PRINCIPAL ARTERIAL  1.000000  
24433  NE 85TH ST    35  PRINCIPAL ARTERIAL  1

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
13663,2273,DIST,3.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.000000
13664,2274,DIST,11.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.500000
13700,2277,DIST,5.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.000000
13705,2280,DIST,5.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,2.000000
23603,3895,DIST,71.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.250000
23607,3896,DIST,7.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,2.000000
23608,3897,DIST,5.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.666667
27225,4561,DIST,7.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.000000
27226,4562,DIST,6.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,2.000000
40780,7394,DIST,2.0,16.0,CONC,1970.0,EvB,10000,NE 85TH ST,35,PRINCIPAL ARTERIAL,1.000000


In [96]:
#No pipes installed on 5th st after 2008, three were pipes installed on 19th ave in 2018
break_2_26_08 = pipes[(pipes["FULL_STRNA"] == "5TH ST") & (pipes["INSTYEAR"] > 2008)]
print(break_2_26_08)
#No pipes installed on 19th ave shared attributes with the broken pipe, only pipes on 5th ST
#Only one pipe was of susbtantial length i.e. >= 5 ft
nearby5 = pipes[(pipes["FULL_STRNA"] == "5TH ST") & (pipes["INSTYEAR"] < 2008) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC")]
nearby5
#End with 1968 as install year

Empty DataFrame
Columns: [FID_WA_Mai, TYPE, LENGTH, DIAMETER, MATERIAL, INSTYEAR, MUSYM, Dist_Rail, FULL_STRNA, SPEED, FUNC_CLASS, SLOPE]
Index: []


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
9478,1613,DIST,1.0,6.0,AC,1977.0,InC,12000,5TH ST,25,NEIGHBORHOOD ACCESS,2.5
9480,1614,DIST,1.0,6.0,AC,1977.0,InC,12000,5TH ST,25,NEIGHBORHOOD ACCESS,3.0
11126,1893,DIST,3.0,6.0,AC,1968.0,AgC,13000,5TH ST,25,NEIGHBORHOOD ACCESS,3.0
23523,3872,DIST,1.0,6.0,AC,1968.0,AmC,13000,5TH ST,25,NEIGHBORHOOD ACCESS,1.0
24467,4071,DIST,20.0,6.0,AC,1968.0,AgC,13000,5TH ST,25,NEIGHBORHOOD ACCESS,2.0


In [110]:
#No pipes on 67th st installed on or after 2008 nor pipes installed before that share attributes with broken pipe
break_4_2_08 = pipes[(pipes["FULL_STRNA"] == "NE 112TH ST")]
print(break_4_2_08)
nearby6 = pipes[(pipes["FULL_STRNA"] == "NE 67TH ST") & (pipes["MATERIAL"] == "AC")]
nearby6


       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
2997          543  DIST     1.0       8.0       DI    1977.0   AgC       5000   
3565          641  DIST   316.0      12.0       DI    1978.0   AgB       7000   
4004          744  DIST     1.0       8.0       DI    1977.0   AgC       5000   
4005          745  DIST    64.0       8.0       DI    1977.0   AgC       5000   
4007          746  DIST   253.0       8.0       DI    1977.0   AgC       5000   
6777         1188  DIST     3.0       8.0       DI    2008.0   AgC      10000   
7092         1236  DIST    41.0       8.0       DI    1990.0   AgB       3000   
7104         1241  DIST   336.0      16.0       DI    1990.0   AgB       3000   
7106         1243  DIST     2.0      16.0       DI    1990.0   AgB       3000   
7107         1244  DIST     1.0      16.0       DI    1990.0   AgB       3000   
7108         1245  DIST   140.0      16.0       DI    1990.0   AgB       3000   
8920         1543  DIST   29

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22251,3672,DIST,44.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.600000
22265,3674,DIST,448.0,6.0,AC,1961.0,AgC,8000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,1.629630
22357,3680,DIST,226.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,1.928571
22371,3681,DIST,6.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.000000
22372,3682,DIST,3.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.000000
22373,3683,DIST,65.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,1.666667
29585,4972,DIST,261.0,6.0,AC,1961.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.200000
29609,4973,DIST,313.0,6.0,AC,1970.0,AgC,9000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.458333
32846,5504,DIST,717.0,6.0,AC,1967.0,AmC,8000,NE 67TH ST,25,NEIGHBORHOOD ACCESS,2.333333


In [122]:
#Finding pipes installed on 73rd st the same year as the break
break_4_4_08 = pipes[(pipes["FULL_STRNA"] == "NE 73RD ST") & (pipes["INSTYEAR"] == 2008)]
print(break_4_4_08)
#Finding earlier installed pipes with the same attributes as the broken pipe
nearby7 = pipes[(pipes["FULL_STRNA"] == "NE 73RD ST") & (pipes["INSTYEAR"] < 2008) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["Dist_Rail"] == 8000)]
nearby7
#Two possible years, 1959 and 1963

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
22567        3719  DIST     3.0       8.0       DI    2008.0   AgC       8000   
22568        3720  DIST     4.0       8.0       DI    2008.0   AgC       8000   
23946        3992  DIST    15.0       8.0       DI    2008.0   AgC       8000   
23948        3993  DIST     3.0       8.0       DI    2008.0   AgC       8000   
26885        4497  DIST     3.0       8.0       DI    2008.0   AgC       8000   
28129        4706  DIST   102.0       8.0       DI    2008.0   AgC       8000   
30433        5100  DIST   482.0       8.0       DI    2008.0   AgC       8000   

       FULL_STRNA SPEED           FUNC_CLASS     SLOPE  
22567  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
22568  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
23946  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
23948  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
26885  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.642857  
28129  NE

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
26886,4498,DIST,644.0,6.0,AC,1959.0,AgC,8000,NE 73RD ST,25,NEIGHBORHOOD ACCESS,2.000000
29078,4884,DIST,461.0,6.0,AC,1963.0,AgC,8000,NE 73RD ST,25,NEIGHBORHOOD ACCESS,0.666667


In [134]:
#Could have used pipes on 2nd st or 5th ave, only pipes installed on or after 2008 on 2nd st 
break_4_18_08 = pipes[(pipes["FULL_STRNA"] == "2ND ST") & (pipes["INSTYEAR"] >= 2008)]
print(break_4_18_08)
#Find pipes installed before 2008 with same attributes as broken pipe
nearby8 = pipes[(pipes["FULL_STRNA"] == "2ND ST") & (pipes["INSTYEAR"] < 2008) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["Dist_Rail"] == 14000)]
nearby8
#Two possible years, 1960 and 1949. Most likely 1949

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
8593         1482  DIST     4.0       8.0       DI    2009.0   AmC      15000   
8611         1484  DIST   265.0       8.0       DI    2009.0   AmC      15000   
8620         1485  DIST     3.0       8.0       DI    2009.0   AmC      15000   
8621         1486  DIST     3.0       8.0       DI    2009.0   AmC      15000   
8622         1487  DIST     3.0       8.0       DI    2009.0   AmC      15000   
8676         1502  DIST    59.0       8.0       DI    2009.0   AmC      15000   
8680         1503  DIST    54.0       8.0       DI    2009.0   AmC      15000   
36531        6146  DIST    52.0       8.0       DI    2009.0   AmC      15000   
39054        6895  DIST   209.0       8.0       DI    2009.0   AmC      14000   

      FULL_STRNA SPEED           FUNC_CLASS     SLOPE  
8593      2ND ST    25  NEIGHBORHOOD ACCESS  2.000000  
8611      2ND ST    25  NEIGHBORHOOD ACCESS  1.555556  
8620      2ND ST    2

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
6354,1097,DIST,23.0,6.0,AC,1960.0,AmC,14000,2ND ST,25,NEIGHBORHOOD ACCESS,1.000000
6356,1099,DIST,39.0,6.0,AC,1949.0,AmC,14000,2ND ST,25,NEIGHBORHOOD ACCESS,2.400000
6397,1107,DIST,42.0,6.0,AC,1949.0,AmC,14000,2ND ST,25,NEIGHBORHOOD ACCESS,2.000000
20843,3450,DIST,488.0,6.0,AC,1949.0,AmC,14000,2ND ST,25,NEIGHBORHOOD ACCESS,1.136364


In [135]:
break_5_2_08 = pipes[(pipes["FULL_STRNA"] == "5TH PL")]
break_5_2_08
#Only six pipes on 5th Pl, relevent pipes do not have install year

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
6909,1219,DIST,766.0,8.0,AC,1977.0,InC,12000,5TH PL,25,COLLECTOR,0.961538
6935,1220,DIST,716.0,8.0,CI,1943.0,InC,12000,5TH PL,25,COLLECTOR,1.074074
6962,1221,DIST,6.0,8.0,CI,1943.0,InC,12000,5TH PL,25,COLLECTOR,2.000000
6963,1222,DIST,1.0,8.0,DI,1977.0,InC,12000,5TH PL,25,COLLECTOR,2.000000
12268,2040,DIST,212.0,4.0,AC,NaN,InC,12000,5TH PL,25,NEIGHBORHOOD ACCESS,0.800000
41013,7482,DIST,2.0,4.0,AC,NaN,InC,12000,5TH PL,25,NEIGHBORHOOD ACCESS,1.333333


In [144]:
#Could have been 1st st or 13th ave, 1st st had installs in 2009, 13th ave did not
break_7_29_08 = pipes[(pipes["FULL_STRNA"] == "1ST ST") & (pipes["INSTYEAR"] == 2009)]
print(break_7_29_08)
#Find pipes that were installed before 2008 with the same attributes at the pipe that broke
nearby10 = pipes[(pipes["FULL_STRNA"] == "1ST ST") & (pipes["INSTYEAR"] < 2009) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["Dist_Rail"] == 15000)]
nearby10
#End with install year of 1960

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
14025        2339  DIST    49.0       8.0       DI    2009.0   AmC      15000   
14028        2340  DIST   171.0       8.0       DI    2009.0   AmC      15000   
20806        3446  DIST     1.0       8.0       DI    2009.0   AmC      15000   
20807        3447  DIST    62.0       8.0       DI    2009.0   AmC      15000   
20937        3460  DIST     1.0       8.0       DI    2009.0   AmC      15000   
20938        3461  DIST     2.0       8.0       DI    2009.0   AmC      15000   

      FULL_STRNA SPEED           FUNC_CLASS     SLOPE  
14025     1ST ST    25  NEIGHBORHOOD ACCESS  1.666667  
14028     1ST ST    25  NEIGHBORHOOD ACCESS  1.800000  
20806     1ST ST    25  NEIGHBORHOOD ACCESS  2.000000  
20807     1ST ST    25  NEIGHBORHOOD ACCESS  1.400000  
20937     1ST ST    25  NEIGHBORHOOD ACCESS  2.000000  
20938     1ST ST    25  NEIGHBORHOOD ACCESS  2.000000  


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
12177,2030,DIST,484.0,6.0,AC,1960.0,AmC,15000,1ST ST,25,NEIGHBORHOOD ACCESS,2.148148


In [155]:
#Find pipes that were installed on 80th st in 2008
break_7_30_08 = pipes[(pipes["FULL_STRNA"] == "NE 80TH ST") & (pipes["INSTYEAR"] == 2008)]
print(break_7_30_08)
#None of the passed installed pipes had the same rail distance but was close, 9000 and 8000 which could theoretically be as close as 7999 and 8000 ft
nearby11 = pipes[(pipes["FULL_STRNA"] == "NE 80TH ST") & (pipes["INSTYEAR"] < 2008) & (pipes["DIAMETER"] == 8) & (pipes["MATERIAL"] == "DI")]
nearby11
#End with install year of 1979

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
18535        3045  DIST    11.4       8.0       DI    2008.0   AgC       9000   
33640        5644  DIST   275.0      20.0       DI    2008.0   AgC       9000   
33648        5645  DIST     6.0      20.0       DI    2008.0   AgC       9000   
33649        5646  DIST   335.0      20.0       DI    2008.0   AgC       9000   
36077        6052  DIST   332.0       8.0       DI    2008.0   AgC       9000   
36211        6081  DIST     6.0      20.0       DI    2008.0   AgC       9000   
36212        6082  DIST     6.0       8.0       DI    2008.0   AgC       9000   
36213        6083  DIST    52.0       8.0       DI    2008.0   AgC       9000   
36215        6084  DIST    28.0       6.0       DI    2008.0   AgC       9000   
36219        6085  DIST     5.5       8.0       DI    2008.0   AgC       9000   

       FULL_STRNA SPEED FUNC_CLASS  SLOPE  
18535  NE 80TH ST    25  COLLECTOR    2.0  
33640  NE 80TH ST   

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
18630,3056,DIST,171.0,8.0,DI,1979.0,AgC,8000,NE 80TH ST,25,COLLECTOR,0.333333
32000,5339,DIST,3.0,8.0,DI,2006.0,AgC,7000,NE 80TH ST,25,COLLECTOR,1.000000
36290,6108,DIST,259.0,8.0,DI,2006.0,AgC,7000,NE 80TH ST,25,COLLECTOR,1.000000


In [165]:
#No pipes installed in 2008 nor after on 119th st, but 110th st does not have pipes that have the same attributes as the pipe that broke
break_9_17_08 = pipes[(pipes["FULL_STRNA"] == "NE 119TH ST")]
break_9_17_08
#End with install year of 1965?

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3970,732,DIST,2.0,12.0,DI,1990.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.000000
3971,733,DIST,3.0,12.0,DI,1990.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.000000
3972,734,DIST,2.0,8.0,DI,1980.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,0.000000
3973,735,DIST,192.0,8.0,DI,1980.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.000000
8297,1418,DIST,295.0,12.0,DI,1990.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.200000
8302,1419,DIST,243.0,8.0,DI,1978.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,2.800000
8312,1420,DIST,56.0,12.0,DI,1990.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,2.000000
8315,1421,DIST,2.0,8.0,DI,1978.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,2.000000
8443,1444,DIST,93.0,8.0,DI,1979.0,InC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.000000
8448,1447,DIST,100.0,8.0,DI,1980.0,AgC,3000,NE 119TH ST,10,NEIGHBORHOOD ACCESS,1.166667
